In [1]:
import numpy as np
import glob as glob
from os.path import join, basename, dirname, exists
import sys
print(sys.executable)  #print kernel path
#print(sys.path)

# tensorflow imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import models
from tensorflow.keras import layers
print(tf.__version__)

# custom imports
import librispect as lspct
from librispect.features import predict
from librispect.utils import split_validation

# cpc import
sys.path.insert(1, '../')
from data_utils import SortedNumberGenerator
sys.path.insert(1, 'notebooks/')

/home/AD/kachiem/miniconda3/envs/tf-conda/bin/python3
1.13.1


attempting to connect srihita's spectrogram function to train_model  
- first, the network itself

In [56]:
def network_encoder(x, code_size):

    ''' Define the network mapping images to embeddings '''

    x = keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, activation='linear')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.LeakyReLU()(x)
    x = keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, activation='linear')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.LeakyReLU()(x)
    x = keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, activation='linear')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.LeakyReLU()(x)
    x = keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, activation='linear')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.LeakyReLU()(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(units=256, activation='linear')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.LeakyReLU()(x)
    x = keras.layers.Dense(units=code_size, activation='linear', name='encoder_embedding')(x)

    return x


def network_autoregressive(x):

    ''' Define the network that integrates information along the sequence '''

    # x = keras.layers.GRU(units=256, return_sequences=True)(x)
    # x = keras.layers.BatchNormalization()(x)
    x = keras.layers.GRU(units=256, return_sequences=False, name='ar_context')(x)

    return x


def network_prediction(context, code_size, predict_terms):

    ''' Define the network mapping context to multiple embeddings '''

    outputs = []
    for i in range(predict_terms):
        outputs.append(keras.layers.Dense(units=code_size, activation="linear", name='z_t_{i}'.format(i=i))(context))

    if len(outputs) == 1:
        output = keras.layers.Lambda(lambda x: K.expand_dims(x, axis=1))(outputs[0])
    else:
        output = keras.layers.Lambda(lambda x: K.stack(x, axis=1))(outputs)

    return output


class CPCLayer(keras.layers.Layer):

    ''' Computes dot product between true and predicted embedding vectors '''

    def __init__(self, **kwargs):
        super(CPCLayer, self).__init__(**kwargs)

    def call(self, inputs):

        # Compute dot product among vectors
        preds, y_encoded = inputs
        dot_product = K.mean(y_encoded * preds, axis=-1)
        dot_product = K.mean(dot_product, axis=-1, keepdims=True)  # average along the temporal dimension

        # Keras loss functions take probabilities
        dot_product_probs = K.sigmoid(dot_product)

        return dot_product_probs

    def compute_output_shape(self, input_shape):
        return (input_shape[0], 1)


def network_cpc(image_shape, terms, predict_terms, code_size, learning_rate):

    ''' Define the CPC network combining encoder and autoregressive model '''

    # Set learning phase (https://stackoverflow.com/questions/42969779/keras-error-you-must-feed-a-value-for-placeholder-tensor-bidirectional-1-keras)
    K.set_learning_phase(1)

    # Define encoder model
    encoder_input = keras.layers.Input(image_shape)
    encoder_output = network_encoder(encoder_input, code_size)
    encoder_model = keras.models.Model(encoder_input, encoder_output, name='encoder')
    encoder_model.summary()

    # Define rest of model
    x_input = keras.layers.Input((terms, image_shape[0], image_shape[1], image_shape[2]))
    x_encoded = keras.layers.TimeDistributed(encoder_model)(x_input)
    context = network_autoregressive(x_encoded)
    preds = network_prediction(context, code_size, predict_terms)

    y_input = keras.layers.Input((predict_terms, image_shape[0], image_shape[1], image_shape[2]))
    y_encoded = keras.layers.TimeDistributed(encoder_model)(y_input)

    # Loss
    dot_product_probs = CPCLayer()([preds, y_encoded])

    # Model
    cpc_model = keras.models.Model(inputs=[x_input, y_input], outputs=dot_product_probs)

    # Compile model
    cpc_model.compile(
        optimizer=keras.optimizers.Adam(lr=learning_rate),
        loss='binary_crossentropy',
        metrics=['binary_accuracy']
    )
    cpc_model.summary()

    return cpc_model




# keep scrollin'!

shaping input for network - spectrogram functions

In [3]:
'''
Instead of images and labels, I have spectrogram segments x and y.
So x is input and y would be output (aka labels). 
You train on bunch of (x - power,y - number of samples), let's call it train_x and train_y.
Then when you input test_x, you should get test_y (or vis_x and vis_y in this case)
'''

# spect_height = number of freq. bins
# y always has one time bin
# n_lags is number of time bins of x
# shape of x = (n_lags,spect_height) x number of samples
# shape of y = (1, spect_height) x number of samples

n_lags = 2
spect_height = 32
NIN = n_lags*spect_height

In [4]:
# location
datafolder = '/home/AD/kachiem/memmap/memmap_dataset_stimulus/'
visfolder = '/home/AD/kachiem/memmap/memmap_dataset_stimulus_vis/'
x_loc = '%sx_lag%03d.dat' % (datafolder, n_lags)
y_loc = '%sy_lag%03d.dat' % (datafolder, n_lags)
x_loc_vis = '%sx_lag%03d.dat' % (visfolder, n_lags)
y_loc_vis = '%sy_lag%03d.dat' % (visfolder, n_lags)

#### creating train data  (cell converted to markdown to prevent running)
y = np.memmap(y_loc, dtype='float32', mode='r')
num_data_samples = int(len(y) / spect_height)

x_data = np.memmap(x_loc, dtype='float32', mode='r+', shape=(n_lags*spect_height, num_data_samples))
y_data = np.memmap(y_loc, dtype='float32', mode='r+', shape=(spect_height, num_data_samples))

x_data.shape, y_data.shape

#### saving train data
np.save('y.npy', y)
np.save('x_data.npy', x_data)
np.save('y_data.npy', y_data)

adapting srihita's spectrogram functions

#### generating test data
y_data_vis = np.memmap(y_loc_vis, dtype='float32', mode='r')
num_data_samples_vis = int(len(y_data_vis) / spect_height)

x_data_vis = np.memmap(x_loc_vis, dtype='float32', mode='r', shape=(n_lags*spect_height, num_data_samples_vis))
y_data_vis = np.memmap(y_loc_vis, dtype='float32', mode='r', shape=(spect_height, num_data_samples_vis))
x_data_vis.shape, y_data_vis.shape

#### saving test data
np.save('y_vis.npy', y_data_vis)
np.save('x_data_vis.npy', x_data_vis)
np.save('y_data_vis.npy', y_data_vis)

In [5]:
# args for the following class, MemmapHandler
datafolder = '/home/AD/kachiem/memmap/memmap_dataset_stimulus/'
visfolder = '/home/AD/kachiem/memmap/memmap_dataset_stimulus_vis/'
n_lags = 2
spect_height = 32
#subset = 'train'

In [6]:
mh = MemmapHandler()

NameError: name 'MemmapHandler' is not defined

In [7]:
# needs WERK
class MemmapHandler(object):
    ''' making and saving memmaps '''
    def __init__(self):
        # args:  folder, n_lags, spect_height, subset
        
        # spect_height = number of freq. bins
        # y always has one time bin
        # n_lags is number of time bins of x
        # shape of x = (n_lags,spect_height) x number of samples
        # shape of y = (1, spect_height) x number of samples
        
        # input variables
        self.n_lags = 2
        self.spect_height = 32
        self.subset = ''  # train/valid/test
        
        # generated variables
        self.NIN = self.n_lags*self.spect_height
        self.train, self.sub_train, self.test, self.sub_test = self.init_data()
        #self.train, self.sub_train, self.valid, self.sub_valid, self.test, self.sub_test = self.init_data()
        self.num_data_samples = self.get_n_samples(self.subset)
        self.shape = (n_lags*spect_height, self.num_data_samples)
        
        def init_data(self):
            # args: folder, name, data
            ''' Create intitial stimulus and stimulus_vis memmaps '''
            from pathlib import Path

            def read_file_loc(self, folder, n_lags):
                ''' function to read in name of .dat file '''
                loc = '%sx_lag%03d.dat' % (folder, n_lags)
                return loc

            def if_file(folder):
                # check if stimulus or stimulus_vis
                if folder.contains('vis'):
                    filename = 'memmap_vis.npy'
                    subset = 'test'
                    f_loc = read_file_loc(folder, self.n_lags)
                else:
                    filename = 'memmap.npy'
                    subset = 'train'
                    f_loc = read_file_loc(folder, self.n_lags)

                filepath = Path(folder + filename)

                # check if file exists, and either load or generate.
                if data_file.is_file():
                    data = load_memmap(filename)
                else:
                    data = generate_memmap(filename, f_loc, shape)
                    sav_data = save_memmap(filepath, f_loc)
                return data, subset
            
            def get_train_set(source='/home/AD/kachiem/memmap/memmap_dataset_stimulus/'):
                train_set, subset = if_file(source, data)
                return train_set, subset

            def get_test_set(source='/home/AD/kachiem/memmap/memmap_dataset_stimulus_vis/'):
                test_set, subset = if_file(source, data)
                return test_set, subset
            
            
            train_set, train_sub = get_train_set()
            test_set, test_sub = get_test_set()
            
            return train_set, train_sub, test_set, test_sub
            
            
        
        def get_n_samples(self, data, subset):
            ''' function to get number of samples in subset '''
            if subset == 'train':
                return int(len(data) / self.spect_height)
            elif subset == 'test':
                return int(len(data) / self.spect_height)
        
        
        def generate_memmap(self, data, shape):
            new_map = np.memmap(data, dtype='float32', mode='r', shape=shape)
            return new_map
        
        def save_memmap(self, folder, filename, data):
            memmap = np.save(folder + filename, data)
            return memmap
            
        def load_memmap(self, filename):
            saved_map = np.lib.format.open_memmap(filename, dtype='float32', mode='r', shape=shape)
            return saved_map
            
        def memmap_copy(self, memmap_in, start_idx, end_idx):
            ''' function to copy memmap and save ram !! '''
            memmap_copy = memmap_in[:]
            subset = memmap_copy[start_idx, end_idx]
            return subset

# start here 

In [8]:
def memmap_copy(self, memmap_in, start_idx, end_idx):
    ''' function to copy memmap and save ram !! '''
    memmap_copy = memmap_in[:]
    subset = memmap_copy[start_idx, end_idx]
    return subset

In [9]:
# loading train data
why = np.lib.format.open_memmap('y.npy', dtype='float32', mode='r')
num_data_samples = int(len(why) / spect_height)
xdata = np.lib.format.open_memmap('x_data.npy', dtype='float32', mode='r', shape=(n_lags*spect_height, num_data_samples))
ydata = np.lib.format.open_memmap('y_data.npy', dtype='float32', mode='r', shape=(spect_height, num_data_samples))

xdata.shape, ydata.shape

((64, 149962), (32, 149962))

In [10]:
# training set: memmap inputs
data = why[:]
x_data = xdata[:, :1000]
y_data = ydata[:, :1000]

In [11]:
# loading test data
why_vis = np.lib.format.open_memmap('y_vis.npy', dtype='float32', mode='r')
num_data_samples_vis = int(len(why_vis) / spect_height)
xdata_vis = np.lib.format.open_memmap('x_data_vis.npy', dtype='float32', mode='r', shape=(n_lags*spect_height, num_data_samples_vis))
ydata_vis = np.lib.format.open_memmap('y_data_vis.npy', dtype='float32', mode='r', shape=(spect_height, num_data_samples_vis))

xdata.shape, ydata.shape

((64, 149962), (32, 149962))

In [12]:
# training set: memmap inputs
data_vis = why_vis[:]
x_data_vis = xdata_vis[:, :1000]
y_data_vis = ydata_vis[:, :1000]

In [13]:
# adapted for test set
class Shuffled_memmap_testset(object):

    def __init__(self, data, x_data, y_data, subset, n_lags, spect_height, vis_ratio=0, valid_ratio=.06):

        # Set params
        self.data = data
        self.x_data = x_data
        self.y_data = y_data
        self.subset = subset
        self.n_lags = n_lags
        self.spect_height = spect_height
        
        self.data_idxs = np.arange(self.num_data_samples)
        self.num_data_samples_vis = int(len(data) / spect_height)
        self.data_idxs_vis = np.arange(self.num_data_samples_vis)
        
        np.random.shuffle(self.data_idxs)
        self.valid_idxs, self.data_idxs = np.split(self.data_idxs, [self.num_valid_samples])
        self.num_data_samples -= self.num_valid_samples

        
        def get_data(self, idx):
            return self.x_data[:, idx].T, self.y_data[:, idx].T

        def data_iterator(self, batch_size=64):
            np.random.shuffle(self.data_idxs)
            for batch_idx in range(0, self.num_data_samples, batch_size):
                shuff_idx = self.data_idxs[batch_idx:batch_idx+batch_size]
                yield self.get_data(shuff_idx)
        
        def vis_set(self):
        #return self.get_data(self.vis_idxs)
            return self.get_data(self.data_idxs_vis)

        #def vis_iterator(self, start=0, end=None, batch_size=1):
        #    if end is None:
        #        end = self.num_vis_samples
        #    for batch_idx in range(start, end, batch_size):
        #        data_idx = self.vis_idxs[batch_idx:batch_idx+batch_size]
        #        yield self.get_data(data_idx)

        def vis_iterator(self, start=0, end=None, batch_size=1):
            if end is None:
                end = self.num_data_samples_vis
            for batch_idx in range(start, end, batch_size):
                data_idx = self.data_idxs_vis[batch_idx:batch_idx+batch_size]
                yield self.get_data(data_idx)


In [66]:
# adapted for subset
class Shuffled_memmap_subset(object):
    
    ''' For creating train/valid/test shuffled memmap dataset '''
    
    def __init__(self, data, x_data, y_data, subset, n_lags, spect_height, vis_ratio=0, valid_ratio=.06):
        
        # Set params
        self.data = data
        self.x_data = x_data
        self.y_data = y_data
        self.subset = subset
        self.n_lags = n_lags
        self.spect_height = spect_height
        self.num_data_samples = int(len(data) / spect_height)
        #self.num_data_samples

        '''
        # Initialize memmap handler
        self.memmap_handler = MemmapHandler(self.folder, self.n_lags, self.spect_height, self.subset)

        # Initialize data
        data_loc = self.memmap_handler.read_file_loc(self.folder)
        data = self.memmap_handler.init_data(self.folder, self.saved_data, x_location)
        self.num_data_samples = self.memmap_handler.get_n_samples('train')
        self.dataset = data.memmap_copy(data, data.shape[0], data.shape[1][:1000])
        '''
        
        
        # get dataset+batching indicies
        #self.num_vis_samples = int(self.num_data_samples*vis_ratio)
        self.num_valid_samples = int(self.num_data_samples*valid_ratio)
        #vis_start = np.random.randint(self.num_data_samples - self.num_vis_samples)
       
        #self.vis_idxs = np.arange(vis_start, vis_start + self.num_vis_samples)
        #self.data_idxs = np.delete(np.arange(self.num_data_samples), self.vis_idxs)
        self.data_idxs = np.arange(self.num_data_samples)
        #self.num_data_samples -= self.num_vis_samples
        self.n_batches = self.get_n_batches()
       
        np.random.shuffle(self.data_idxs)
        self.valid_idxs, self.data_idxs = np.split(self.data_idxs, [self.num_valid_samples])
        self.num_data_samples -= self.num_valid_samples
    
    def __len__(self):
        return self.n_batches
    
    def get_n_batches(self, batch_size=64):
        n_batches = 0
        for batch_idx in range(0, self.num_data_samples, batch_size):
            n_batches +=1
        return n_batches
            
    def get_num_samples(self, subset):
        ''' 
        Return number of samples wrt subset. 
            subset: a string - 'train', 'valid', or 'test'
        '''
        if subset == 'train':
            return self.num_data_samples
        elif subset == 'valid':
            return self.num_valid_samples
        elif subset == 'test':
            return self.num_data_samples_vis
    
    def get_data(self, idx):
        return self.x_data[:, idx].T, self.y_data[:, idx].T
   
    def data_iterator(self, batch_size=64):
        np.random.shuffle(self.data_idxs)
        for batch_idx in range(0, self.num_data_samples, batch_size):
            shuff_idx = self.data_idxs[batch_idx:batch_idx+batch_size]
            
            yield self.get_data(shuff_idx)
   
    def valid_set(self):
        return self.get_data(self.valid_idxs)
   
    def valid_iterator(self, batch_size=64):
        for batch_idx in range(0, self.num_valid_samples, batch_size):
            valid_idx = self.valid_idxs[batch_idx:batch_idx+batch_size]
            yield self.get_data(valid_idx)
   


In [67]:
train_data = Shuffled_memmap_subset(
    data=data,
    x_data=x_data,
    y_data=y_data,
    subset='train', 
    n_lags=2,
    spect_height=32
    )

In [68]:
test_data = Shuffled_memmap_subset(
    data=data_vis,
    x_data=x_data_vis,
    y_data=y_data_vis,
    subset='test', 
    n_lags=2,
    spect_height=32
    )

In [65]:
# the OG
class Shuffled_memmap_dataset(object):
    def __init__(self, xfile, yfile, xfile_vis, yfile_vis, vis_ratio=0, valid_ratio=.06):
        y_data = np.memmap(yfile, dtype='float32', mode='r')
        self.num_data_samples = int(len(y_data) / spect_height)
       
        y_data_vis = np.memmap(yfile_vis, dtype='float32', mode='r')
        self.num_data_samples_vis = int(len(y_data_vis) / spect_height)
       
        self.x_data = np.memmap(xfile, dtype='float32', mode='r', shape=(n_lags*spect_height, self.num_data_samples))
        self.y_data = np.memmap(yfile, dtype='float32', mode='r', shape=(spect_height, self.num_data_samples))
       
        self.x_data_vis = np.memmap(xfile_vis, dtype='float32', mode='r', shape=(n_lags*spect_height, self.num_data_samples_vis))
        self.y_data_vis = np.memmap(yfile_vis, dtype='float32', mode='r', shape=(spect_height, self.num_data_samples_vis))
               
        #self.num_vis_samples = int(self.num_data_samples*vis_ratio)
        self.num_valid_samples = int(self.num_data_samples*valid_ratio)
        #vis_start = np.random.randint(self.num_data_samples - self.num_vis_samples)
       
        #self.vis_idxs = np.arange(vis_start, vis_start + self.num_vis_samples)
        #self.data_idxs = np.delete(np.arange(self.num_data_samples), self.vis_idxs)
        self.data_idxs_vis = np.arange(self.num_data_samples_vis)
        self.data_idxs = np.arange(self.num_data_samples)
        #self.num_data_samples -= self.num_vis_samples
       
        np.random.shuffle(self.data_idxs)
        self.valid_idxs, self.data_idxs = np.split(self.data_idxs, [self.num_valid_samples])
        self.num_data_samples -= self.num_valid_samples
           
    def get_data(self, idx):
        return self.x_data[:, idx].T, self.y_data[:, idx].T
   
    def data_iterator(self, batch_size=64):
        np.random.shuffle(self.data_idxs)
        for batch_idx in range(0, self.num_data_samples, batch_size):
            shuff_idx = self.data_idxs[batch_idx:batch_idx+batch_size]
            yield self.get_data(shuff_idx)
   
    def valid_set(self):
        return self.get_data(self.valid_idxs)
   
    def valid_iterator(self, batch_size=64):
        for batch_idx in range(0, self.num_valid_samples, batch_size):
            valid_idx = self.valid_idxs[batch_idx:batch_idx+batch_size]
            yield self.get_data(valid_idx)
   
    def vis_set(self):
        #return self.get_data(self.vis_idxs)
        return self.get_data(self.data_idxs_vis)
   
    #def vis_iterator(self, start=0, end=None, batch_size=1):
    #    if end is None:
    #        end = self.num_vis_samples
    #    for batch_idx in range(start, end, batch_size):
    #        data_idx = self.vis_idxs[batch_idx:batch_idx+batch_size]
    #        yield self.get_data(data_idx)
           
    def vis_iterator(self, start=0, end=None, batch_size=1):
        if end is None:
            end = self.num_data_samples_vis
        for batch_idx in range(start, end, batch_size):
            data_idx = self.data_idxs_vis[batch_idx:batch_idx+batch_size]
            yield self.get_data(data_idx)

#dataset = Shuffled_memmap_dataset(x_loc, y_loc, x_loc_vis, y_loc_vis)

valid_data = Shuffled_memmap_subset(
    folder='/home/AD/kachiem/memmap/memmap_dataset_stimulus/', 
    subset='valid', 
    saved_data,
    n_lags=2,
    spect_height=32
    )

test_data = Shuffled_memmap_subset(
    folder='/home/AD/kachiem/memmap/memmap_dataset_stimulus_vis/', 
    subset='train', 
    saved_data,
    n_lags=2,
    spect_height=32
    )

#### Get visualization dataset
vis_x, vis_y = dataset.vis_set()  #(16653, 64), (16653, 32)

### took a subset of test set for visualization
start = 3000
end = start + 1000
sub_vis_x = vis_x[start:end, :]   #(1000, 64)
sub_vis_y = vis_y[start:end, :]   #(1000, 32) 

Adapting MNIST classes to memmap iterator

In [18]:
# doesn't really work right now
class SortedMemmapGenerator(object):

    ''' 
    Data generator providing lists of sorted memmap 
    '''

    def __init__(batch_size, subset, terms, positive_samples=1, predict_terms=1, image_size=28, color=False, rescale=True):

        # Set params
        self.positive_samples = positive_samples
        self.predict_terms = predict_terms
        self.batch_size = batch_size
        self.terms = terms
        self.image_size = image_size
        self.color = color
        self.rescale = rescale

        # Initialize memmap dataset
        self.train_data = Shuffled_memmap_subset(
            data=data,
            x_data=x_data,
            y_data=y_data,
            subset='train', 
            n_lags=2,
            spect_height=32
            )

        test_data = Shuffled_memmap_subset(
            data=data_vis,
            x_data=x_data_vis,
            y_data=y_data_vis,
            subset='test', 
            n_lags=2,
            spect_height=32
            )
        
        # get num_samples and num_batches
        self.n_samples = dataset.get_num_samples(subset) 
        self.n_batches = self.n_samples // batch_size
        
        #self.n_samples = self.mnist_handler.get_n_samples(subset) // terms
        #self.n_batches = self.n_samples // batch_size

    def __iter__(self):
        return self

    def __next__(self):
        return self.next()

    def __len__(self):
        return self.n_batches

    def next(self):

        # Build sentences
        image_labels = np.zeros((self.batch_size, self.terms + self.predict_terms))
        sentence_labels = np.ones((self.batch_size, 1)).astype('int32')
        positive_samples_n = self.positive_samples 
        for b in range(self.batch_size):

            # Set ordered predictions for positive samples
            seed = np.random.randint(0, 10)
            sentence = np.mod(np.arange(seed, seed + self.terms + self.predict_terms), 10)

            if positive_samples_n <= 0:

                # Set random predictions for negative samples
                # Each predicted term draws a number from a distribution that excludes itself
                numbers = np.arange(0, 10)
                predicted_terms = sentence[-self.predict_terms:]
                for i, p in enumerate(predicted_terms):
                    predicted_terms[i] = np.random.choice(numbers[numbers != p], 1)
                sentence[-self.predict_terms:] = np.mod(predicted_terms, 10)
                sentence_labels[b, :] = 0

            # Save sentence
            image_labels[b, :] = sentence

            positive_samples_n -= 1

        # Retrieve actual images (64, 64, 3) => (32, 2)
        #images, _ = self.mnist_handler.get_batch_by_labels(self.subset, image_labels.flatten(), self.image_size, self.color, self.rescale)
        images, _ = self.dataset.data_iterator(self.n_samples)
        print(images)
        
        # Assemble batch
        images = images.reshape((self.batch_size, self.terms + self.predict_terms, images.shape[1], images.shape[2]))
        # (8, 8, 64, 64, 3)
        x_images = images[:, :-self.predict_terms, ...]   # (8, 4, 64, 64, 3)
        y_images = images[:, -self.predict_terms:, ...]   # (8, 4, 64, 64, 3)

        # Randomize
        idxs = np.random.choice(sentence_labels.shape[0], sentence_labels.shape[0], replace=False)
        # shape: (8,)
        
        return [x_images[idxs, ...], y_images[idxs, ...]], sentence_labels[idxs, ...]


training model

In [69]:
# memmap ver
def train_model(data, x_data, y_data, data_vis, x_data_vis, y_data_vis, n_lags, spect_height, epochs, batch_size, output_dir, code_size, lr=2e-4, terms=3, predict_terms=3, color=True):

    # Prepare data
    # clean up generator more to work by subset
    train_data = Shuffled_memmap_subset(
        data=data,
        x_data=x_data,
        y_data=y_data,
        subset='train', 
        n_lags=n_lags,
        spect_height=spect_height
        )

    test_data = Shuffled_memmap_subset(
        data=data_vis,
        x_data=x_data_vis,
        y_data=y_data_vis,
        subset='test', 
        n_lags=n_lags,
        spect_height=spect_height
        )

    # Prepares the model
    model = network_cpc(spect_height=spect_height, n_lags=n_lags, terms=terms, predict_terms=predict_terms,
                        code_size=code_size, input_shape=(spect_height, n_lags), learning_rate=lr)

    # Callbacks
    history = keras.callbacks.History()
    reduce_LR_Plateau = keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', 
        factor=1/3, 
        patience=2, 
        min_lr=1e-4)
    #callbacks = [history, reduce_LR_Plateau]
    callbacks = [history]
    
    # Trains the model
    modeled = model.fit_generator(
        generator=train_data,
        steps_per_epoch=len(train_data),
        validation_data=test_data,
        validation_steps=len(test_data),
        epochs=epochs,
        verbose=1,
        callbacks=callbacks
    )

    # Saves the model
    # Remember to add custom_objects={'CPCLayer': CPCLayer} to load_model when loading from disk
    model.save(join(output_dir, 'mem_cpc.h5'))

    # Saves the encoder alone
    encoder = model.layers[1].layer
    encoder.save(join(output_dir, 'mem_encoder.h5'))
    
    # plotting loss
    train_loss = modeled.history['loss']
    val_loss = modeled.history['val_loss']
    epoch_count = range(1, epochs+1)    # 10 epochs

    plt.plot(epoch_count, train_loss, 'r--')
    plt.plot(epoch_count, val_loss, 'o-')
    plt.legend(['Training Loss', 'Validation Loss'])
    plt.title('')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    
    t = time.localtime()
    timestamp = time.strftime('%b-%d-%Y_%H%M', t)
    plt.savefig(output_dir + "img/" + timestamp + "_mem.png")

    return model

In [70]:
def network_encoder(x, code_size):

    ''' Define the network mapping images to embeddings '''
    print(x.shape)
    x = keras.layers.Conv1D(filters=2, kernel_size=2, strides=1, data_format="channels_last", activation='relu', input_shape=(32, 2))(x)
    print("conv1d", x.shape)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.LeakyReLU()(x)
    x = keras.layers.Conv1D(filters=2, kernel_size=2, strides=1, activation='relu')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.LeakyReLU()(x)
    x = keras.layers.Conv1D(filters=4, kernel_size=2, strides=2, activation='relu')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.LeakyReLU()(x)
    x = keras.layers.Conv1D(filters=4, kernel_size=2, strides=2, activation='relu')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.LeakyReLU()(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(units=512, activation='relu')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.LeakyReLU()(x)
    x = keras.layers.Dense(units=code_size, activation='relu', name='encoder_embedding')(x)

    return x


def network_autoregressive(x):

    ''' Define the network that integrates information along the sequence '''

    # x = keras.layers.GRU(units=256, return_sequences=True)(x)
    # x = keras.layers.BatchNormalization()(x)
    x = keras.layers.GRU(units=256, return_sequences=False, name='ar_context')(x)

    return x


def network_prediction(context, code_size, predict_terms):

    ''' Define the network mapping context to multiple embeddings '''

    outputs = []
    for i in range(predict_terms):
        outputs.append(keras.layers.Dense(units=code_size, activation="linear", name='z_t_{i}'.format(i=i))(context))

    if len(outputs) == 1:
        output = keras.layers.Lambda(lambda x: K.expand_dims(x, axis=1))(outputs[0])
    else:
        output = keras.layers.Lambda(lambda x: K.stack(x, axis=1))(outputs)

    return output


def network_cpc(spect_height, n_lags, terms, predict_terms, code_size, input_shape, learning_rate):

    ''' Define the CPC network combining encoder and autoregressive model '''

    # Set learning phase (https://stackoverflow.com/questions/42969779/keras-error-you-must-feed-a-value-for-placeholder-tensor-bidirectional-1-keras)
    K.set_learning_phase(1)

    # Define encoder model
    encoder_input = keras.layers.Input(shape=(spect_height, n_lags))
    #print(encoder_input.shape)
    encoder_output = network_encoder(encoder_input, code_size)
    #print(encoder_output.shape)
    encoder_model = keras.models.Model(encoder_input, encoder_output, name='encoder')
    encoder_model.summary()

    # Define rest of model
    x_input = keras.layers.Input((terms, spect_height, n_lags))
    x_encoded = keras.layers.TimeDistributed(encoder_model)(x_input)
    context = network_autoregressive(x_encoded)
    preds = network_prediction(context, code_size, predict_terms)

    y_input = keras.layers.Input((predict_terms, spect_height, n_lags))
    y_encoded = keras.layers.TimeDistributed(encoder_model)(y_input)

    # Loss
    dot_product_probs = CPCLayer()([preds, y_encoded])

    # Model
    cpc_model = keras.models.Model(inputs=[x_input, y_input], outputs=dot_product_probs)

    # Compile model
    cpc_model.compile(
        optimizer=keras.optimizers.Adam(lr=learning_rate),
        loss='binary_crossentropy',
        metrics=['binary_accuracy']
    )
    cpc_model.summary()

    return cpc_model

In [71]:
train_model(
    # x/y train data
    data,
    x_data,
    y_data,
    # x/y test data
    data_vis,
    x_data_vis,
    y_data_vis,
    
    # params
    2, 
    32, 
    10, 8, 'models/memmap', 512)

(?, 32, 2)
conv1d (?, 31, 2)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_25 (InputLayer)        (None, 32, 2)             0         
_________________________________________________________________
conv1d_31 (Conv1D)           (None, 31, 2)             10        
_________________________________________________________________
batch_normalization_v1_35 (B (None, 31, 2)             8         
_________________________________________________________________
leaky_re_lu_35 (LeakyReLU)   (None, 31, 2)             0         
_________________________________________________________________
conv1d_32 (Conv1D)           (None, 30, 2)             10        
_________________________________________________________________
batch_normalization_v1_36 (B (None, 30, 2)             8         
_________________________________________________________________
leaky_re_lu_36 (LeakyReLU)   (None, 30, 2)     

AttributeError: 'Shuffled_memmap_subset' object has no attribute 'shape'

In [31]:
train_model(
    # x/y train data
    data=data,
    x_data=x_data,
    y_data=y_data,
    # x/y test data
    data_vis=data_vis,
    x_data_vis=x_data_vis,
    y_data_vis=y_data_vis,
    
    # params
    n_lags=2, 
    spect_height=32,
    epochs=10,
    batch_size=8,
    output_dir='models/memmap',
    code_size=512,
    lr=2e-4,
    terms=3,
    predict_terms=3,
    color=True
)
    

TypeError: train_model() got an unexpected keyword argument 'n_lags'

In [26]:
# loss fxn
def train_model(x_data, y_data, x_loc_vis, y_loc_vis, epochs, batch_size, output_dir, code_size, lr=1e-4, terms=4, predict_terms=4, image_size=28, color=False):

    # Prepare data
    # clean up generator more to work by subset
    train_data = SortedMemmapGenerator(x_data, y_data, x_loc_vis, y_loc_vis, batch_size=batch_size, subset='train', terms=terms,
                                       positive_samples=batch_size // 2, predict_terms=predict_terms,
                                       image_size=image_size, color=color, rescale=True)

    validation_data = SortedMemmapGenerator(x_data, y_data, x_loc_vis, y_loc_vis, batch_size=batch_size, subset='valid', terms=terms,
                                            positive_samples=batch_size // 2, predict_terms=predict_terms,
                                            image_size=image_size, color=color, rescale=True)

    
    # Prepares the model
    model = network_cpc(image_shape=(image_size, image_size, 3), terms=terms, predict_terms=predict_terms,
                        code_size=code_size, learning_rate=lr)

    # Callbacks
    history = keras.callbacks.History()
    reduce_LR_Plateau = keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', 
        factor=1/3, 
        patience=2, 
        min_lr=1e-4)
    #callbacks = [history, reduce_LR_Plateau]
    callbacks = [history]
    
    # Trains the model
    modeled = model.fit_generator(
        generator=train_data,
        steps_per_epoch=len(train_data),
        validation_data=validation_data,
        validation_steps=len(validation_data),
        epochs=epochs,
        verbose=1,
        callbacks=callbacks
    )

    # Saves the model
    # Remember to add custom_objects={'CPCLayer': CPCLayer} to load_model when loading from disk
    model.save(join(output_dir, 'mem_cpc.h5'))

    # Saves the encoder alone
    encoder = model.layers[1].layer
    encoder.save(join(output_dir, 'mem_encoder.h5'))
    
    # plotting loss
    train_loss = modeled.history['loss']
    val_loss = modeled.history['val_loss']
    epoch_count = range(1, epochs+1)    # 10 epochs

    plt.plot(epoch_count, train_loss, 'r--')
    plt.plot(epoch_count, val_loss, 'o-')
    plt.legend(['Training Loss', 'Validation Loss'])
    plt.title('')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    
    t = time.localtime()
    timestamp = time.strftime('%b-%d-%Y_%H%M', t)
    plt.savefig(output_dir + "img/" + timestamp + "_mem.png")

    return model

main( ) in train_model.py

In [ ]:
## cpc train_model() params / main()
SPECT_HEIGHT = 2048
epochs = 10
batch_size = 32
output_dir = "main"
code_size = 8
hparams = lspct.features.spectrogram.HPARAMS

#path_list = glob.glob((lspct.paths.WAV_DIR / "*.wav").as_posix())[0:19]    # get list of wavs
#training_path_list, validation_path_list = split_validation(path_list, 0.1) # split train and val


In [ ]:
train_model(
    x_data, y_data, x_loc_vis, y_loc_vis,
    epochs=10,
    batch_size=32,
    output_dir='models/memmap',
    code_size=128,
    lr=1e-3,
    terms=4,
    predict_terms=4,
    image_size=64,
    color=True
    
)